In [1]:
import pandas as pd
from statsmodels.multivariate.manova import MANOVA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import KMeans
from xgboost import plot_importance
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
from statsmodels.multivariate.manova import MANOVA
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.impute import SimpleImputer

random.seed(42)
np.random.seed(42)

In [3]:
data = pd.read_excel('BD_numericas.xlsx')

In [ ]:
df = data

In [ ]:
df.head()

In [ ]:
df['MORA'] = (df['DIAS DE MORA'] >= 30).astype(int)


In [ ]:
df.columns = df.columns.str.replace(" ", "_")

In [ ]:
variables_manova = [
    'SCORE',
    'MONTO_DESEMBOLSO',
    'EDAD',
    'NIVEL_ESTUDIOS',
    'INGRESOS_FIJOS',
    'GASTOS_DE_SOSTENIMIENTO',
    'ACTIVOS',
    'TIEMPO_ACTIVIDAD',
    'TIEMPO_EN_RESIDENCIA',
    'CODIGO_CIIU',
    'ESTADOCIVIL',
    'ESTRATO'
]




In [ ]:
df['TOTAL_INGRESOS']

In [ ]:
scaler = StandardScaler()
df_scaled = df.copy()

df_scaled[variables_manova] = scaler.fit_transform(df[variables_manova])

maov_scaled = MANOVA.from_formula(
    "SCORE + INGRESOS_FIJOS + TOTAL_INGRESOS + TIEMPO_ACTIVIDAD + TIEMPO_EN_RESIDENCIA + EDAD + NIVEL_ESTUDIOS + ACTIVOS + GASTOS_DE_SOSTENIMIENTO + CODIGO_CIIU + ESTADOCIVIL + ESTRATO ~ MORA",
    data=df_scaled
)

print(maov_scaled.mv_test())

LDA

In [ ]:

# Imputación de NA
imp = SimpleImputer(strategy="median")
X = imp.fit_transform(df[variables_manova])
y = df["MORA"]

# Balanceo con SMOTE
smote = SMOTE(random_state=42)
X_bal, y_bal = smote.fit_resample(X, y)

#Entrenamiento del modelo
lda = LinearDiscriminantAnalysis()
lda.fit(X_bal, y_bal)

y_pred = lda.predict(X)

print("\n Accuracy LDA:", accuracy_score(y, y_pred))
print("\n Classification Report:\n", classification_report(y, y_pred))
print("\n Matriz de Confusión:\n", confusion_matrix(y, y_pred))


Importancia de las variables

In [ ]:
coef = pd.DataFrame({
    "Variable": variables_manova,
    "Peso_LDA": lda.coef_[0]
}).sort_values(by="Peso_LDA", ascending=False)

print("\n Ranking de importancia LDA:\n")
print(coef)

In [ ]:
df_plot = pd.DataFrame({"LDA1": lda.transform(X)[:,0], "MORA": y})

sns.kdeplot(data=df_plot, x="LDA1", hue="MORA", fill=True, alpha=0.4)
plt.axvline(0, linestyle="--", color="black")
plt.title("Separación entre morosos y no morosos – LDA1")
plt.show()

Umbrales para LDA

In [ ]:
y_proba_lda = lda.predict_proba(X)[:,1]

# Probando umbrales de 0.1 a 0.9
thresholds = [i/100 for i in range(5,95,5)]
resultados = []

for t in thresholds:
    y_pred_t = (y_proba_lda >= t).astype(int)
    resultados.append([
        t,
        precision_score(y, y_pred_t),
        recall_score(y, y_pred_t),
        f1_score(y, y_pred_t)
    ])

import pandas as pd
tabla_lda = pd.DataFrame(resultados, columns=["threshold","precision","recall","f1"])
print(tabla_lda)

In [ ]:
# Probabilidades LDA ya entrenado
y_proba_lda = lda.predict_proba(X)[:,1]

# Cambiando treshold
threshold = 0.45
y_pred_045 = (y_proba_lda >= threshold).astype(int)

print(f"\n Resultados con Threshold = {threshold}")
print("Accuracy:", accuracy_score(y, y_pred_045))
print("\nClassification Report:\n", classification_report(y, y_pred_045))
print("\nConfusion Matrix:\n", confusion_matrix(y, y_pred_045))

In [ ]:
coef = pd.DataFrame({
    "Variable": variables_manova,
    "Peso_LDA": lda.coef_[0]
})

coef["|Peso|"] = coef["Peso_LDA"].abs()
coef.sort_values(by="|Peso|", ascending=False, inplace=True)

print("\n Ranking de variables LDA:\n")
print(coef.drop(columns="|Peso|"))

XGBOOST

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

imp = SimpleImputer(strategy="median")
X_train = imp.fit_transform(X_train)
X_test  = imp.transform(X_test)

model = XGBClassifier(
    learning_rate=0.05,
    max_depth=5,
    n_estimators=300,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))

In [ ]:
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)
model.fit(X_train_bal, y_train_bal)

In [ ]:
y_pred_bal = model.predict(X_test)
y_proba_bal = model.predict_proba(X_test)[:,1]

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

print("\n Resultados XGBoost + SMOTE")
print(classification_report(y_test, y_pred_bal))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_bal))
print("AUC:", roc_auc_score(y_test, y_proba_bal))

Seleccion del umbral

In [ ]:
def evaluar_threshold_smote(thresholds, y_test, y_prob):
    resultados = []
    for t in thresholds:
        y_pred = (y_prob >= t).astype(int)
        resultados.append([
            t,
            precision_score(y_test, y_pred),
            recall_score(y_test, y_pred),
            f1_score(y_test, y_pred)
        ])
    return pd.DataFrame(resultados, columns=["threshold","precision","recall","f1"])


thresholds = np.arange(0.10, 0.91, 0.05)
tabla_t = evaluar_threshold_smote(thresholds, y_test, y_proba_bal)
print(tabla_t)

In [ ]:
threshold = 0.15
y_pred_thr = (y_proba_bal >= threshold).astype(int)

print(f"\n* Resultados con threshold = {threshold} *")
print("Accuracy:", accuracy_score(y_test, y_pred_thr))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_thr))
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_thr))

In [ ]:
auc = roc_auc_score(y_test, y_proba_bal)
print(f"\nAUC ROC del modelo: {auc:.4f}")

Kmeans

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
vars_segmento = [
    "SCORE","CUOTAMENSUAL","INGRESOS_FIJOS",
    "TIEMPO_ACTIVIDAD","TIEMPO_EN_RESIDENCIA",
    "EDAD","TASA","PLAZO"
]

X_seg = df[vars_segmento]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_seg)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

for k in range(2,7):
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(X_pca)
    print(f"k={k} → Silhouette={silhouette_score(X_pca, labels):.3f}")


In [ ]:
# Elegimos el mayor sil
k_optimo = 3
kmeans = KMeans(n_clusters=k_optimo, random_state=42)
df["cluster_final"] = kmeans.fit_predict(X_pca)

In [ ]:
cluster_summary = df.groupby("cluster_final")[vars_segmento].mean()
print(cluster_summary)

Tamaño de clusters

In [ ]:
df["cluster_final"].value_counts()

In [ ]:
df.groupby("cluster_final")[vars_segmento].median()

In [ ]:
df[vars_segmento+["cluster_final"]].groupby("cluster_final").describe()

In [ ]:
plt.figure(figsize=(7,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=df["cluster_final"], cmap="viridis", alpha=0.6)
plt.title("Clusters usando PCA + KMeans")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(label="Cluster")
plt.show()

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1','PC2'],
    index=vars_segmento
)
loadings

In [ ]:
plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=df["cluster_final"], cmap="viridis", alpha=0.6)
plt.colorbar(scatter, label="Cluster")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Clusters con vectores PCA")

for i, var in enumerate(loadings.index):
    plt.arrow(0,0, loadings.PC1[i]*3, loadings.PC2[i]*3, color='red', width=0.01)
    plt.text(loadings.PC1[i]*3.2, loadings.PC2[i]*3.2, var, color='red')

plt.show()

Robustez

In [ ]:
from sklearn.metrics import classification_report

df_test = df.loc[y_test.index].copy()

subgroups = {
    "Género": "GÉNERO",
    "Estrato": "ESTRATO",
    "Nivel de estudios": "NIVEL_ESTUDIOS",
}

for nombre, col in subgroups.items():
    print(f"\n== Robustez por {nombre} ({col}) ==")

    for valor in sorted(df_test[col].dropna().unique()):
        mask = df_test[col] == valor

        if mask.sum() < 100:
            continue

        print(f"\nSubgrupo: {valor}  (n = {mask.sum()})")
        print(classification_report(y_test[mask], y_pred_thr[mask], digits=2))